In [15]:
!pip install tensorflow
!pip install spacy

     ---------------------------------------- 0.0/45.9 kB ? eta -:--:--
     -------- ------------------------------- 10.2/45.9 kB ? eta -:--:--
     --------------------------------- ---- 41.0/45.9 kB 495.5 kB/s eta 0:00:01
     -------------------------------------- 45.9/45.9 kB 456.7 kB/s eta 0:00:00
     ---------------------------------------- 0.0/65.2 kB ? eta -:--:--
     ---------------------------------------- 65.2/65.2 kB 1.8 MB/s eta 0:00:00
     ---------------------------------------- 0.0/181.6 kB ? eta -:--:--
     ----------------- --------------------- 81.9/181.6 kB 1.6 MB/s eta 0:00:01
     ------------------------------------ - 174.1/181.6 kB 2.1 MB/s eta 0:00:01
     -------------------------------------- 181.6/181.6 kB 2.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/12.1 MB ? eta -:--:--
    --------------------------------------- 0.2/12.1 MB 6.3 MB/s eta 0:00:02
   - -------------------------------------- 0.4/12.1 MB 4.9 MB/s eta 0:00:03
   - -

In [75]:
##Tried GloVe Embedding 300 dimension
 
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SpatialDropout1D, LSTM, Dense, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from nltk.stem import WordNetLemmatizer
 
# Download and load nltk stopwords
nltk.download('stopwords')
 
# Read the dataset
data = pd.read_csv('products_details_labelled_complete.csv')
data['column3'] = data['product_title'] + " " + data['description']
import spacy

# Load SpaCy's English tokenizer, tagger, parser, NER, and word vectors
nlp = spacy.load('en_core_web_sm')

def clean_text_spacy(text):
    text = str(text)
    text = text.lower()
    text = re.sub('[/(){}\[\]\|@,;]', ' ', text)
    text = re.sub('[^0-9a-z #+_]', '', text)

    # Create a SpaCy document for further processing
    doc = nlp(text)

    # Lemmatization and stopword removal
    text = ' '.join(token.lemma_ for token in doc if token.text not in stopwords.words('english'))

    return text

# Apply text preprocessing
data['column3'] = data['column3'].apply(clean_text_spacy)
print(data['column3'].head())
token_list=[]
tokenized_list=data['column3'].tolist()
for seq in tokenized_list:
    token_list.append(seq.split())

[nltk_data] Downloading package stopwords to C:\Users\Zeeshan
[nltk_data]     Gilani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0    philips recpironics breathing gadget like new ...
1    foam roller 36 white white long roller leave g...
2    kolua wax   large package hair removal   full ...
3    yoga deck 50 pose   meditation body mind   spi...
4    medical rolling portable fold adult mobility w...
Name: column3, dtype: object


In [76]:
print(token_list[:10])

[['philips', 'recpironics', 'breathing', 'gadget', 'like', 'new', 'gently', 'use', 'problem', 'clean', 'neat', 'ready', 'use'], ['foam', 'roller', '36', 'white', 'white', 'long', 'roller', 'leave', 'great', 'condition', 'sunset', 'parkroller', 'yoga'], ['kolua', 'wax', 'large', 'package', 'hair', 'removal', 'full', 'try', 'rather', 'go', 'spa', 'work', 'great'], ['yoga', 'deck', '50', 'pose', 'meditation', 'body', 'mind', 'spirit', 'card', 'yoga', 'deck', '50', 'pose', 'meditation', 'body', 'mind', 'spirit', 'cardsall', 'hereflashcard', 'really', 'help', 'pose'], ['medical', 'rolling', 'portable', 'fold', 'adult', 'mobility', 'walker', '2', 'wheel', 'good', 'conditioncome', 'pet', 'free', 'smoke', 'free', 'home', 'locate', 'midwood', 'brooklyn', 'ocean', 'ave', 'avenue', 'j', 'avenue', 'j', 'station', 'q', 'b', 'train', 'line', 'brooklyn', 'call', 'text', '718', '5139877', 'click', 'reply', 'button', 'top'], ['calvin', 'klein', 'eyeglass', 'use', 'good', 'good', 'condition', 'sale', 'a

In [20]:
 
# Text Preprocessing Function
def clean_text(text):
    text = text.lower()
    text = re.sub('[/(){}\[\]\|@,;]', ' ', text)
    text = re.sub('[^0-9a-z #+_]', '', text)
    text = ' '.join(word for word in text.split() if word not in stopwords.words('english'))
    lemmatizer = WordNetLemmatizer()
    text = ' '.join(lemmatizer.lemmatize(word) for word in text.split())  # Lemmatization
    return text
# Apply text preprocessing
data['product_title'] = data['product_title'].apply(clean_text)
tokenized_list=data['product_title'].tolist()
token_list=tokenized_list.split()

In [17]:
!python -m spacy download en_core_web_sm


     ---------------------------------------- 0.0/12.8 MB ? eta -:--:--
     ---------------------------------------- 0.1/12.8 MB 3.2 MB/s eta 0:00:04
      --------------------------------------- 0.2/12.8 MB 3.5 MB/s eta 0:00:04
     - -------------------------------------- 0.4/12.8 MB 4.0 MB/s eta 0:00:04
     -- ------------------------------------- 0.7/12.8 MB 4.7 MB/s eta 0:00:03
     --- ------------------------------------ 1.0/12.8 MB 5.3 MB/s eta 0:00:03
     --- ------------------------------------ 1.2/12.8 MB 4.8 MB/s eta 0:00:03
     ---- ----------------------------------- 1.4/12.8 MB 4.8 MB/s eta 0:00:03
     ---- ----------------------------------- 1.5/12.8 MB 4.3 MB/s eta 0:00:03
     ----- ---------------------------------- 1.8/12.8 MB 4.5 MB/s eta 0:00:03
     ------ --------------------------------- 2.0/12.8 MB 4.5 MB/s eta 0:00:03
     ------ --------------------------------- 2.1/12.8 MB 4.5 MB/s eta 0:00:03
     ------- -------------------------------- 2.3/12.8 MB 4

In [73]:
# Tokenization and Vectorization
tokenizer = Tokenizer(num_words=50000, lower=True)
tokenizer.fit_on_texts(data['column3'])
X = tokenizer.texts_to_sequences(data['column3'])

X = pad_sequences(X, maxlen=250)
 
# Convert labels to one-hot encoding
Y = pd.get_dummies(data['Label']).values
 
# Train-test split
X_train, X_test, y_train, y_test, indices_train, indices_test = train_test_split(X,Y, range(data.shape[0]), test_size=0.3, random_state=42, shuffle=True
)
print(X[:10])
print(X_train.shape)

[[1508, 6336, 1812, 4114, 58, 1, 615, 2, 789, 48, 1634, 898, 2], [105, 468, 1007, 178, 178, 109, 468, 222, 59, 40, 4115, 6337, 469], [6338, 275, 98, 221, 10, 470, 92, 865, 3249, 144, 250, 53, 59], [469, 1635, 71, 2687, 790, 23, 1008, 3250, 791, 469, 1635, 71, 2687, 790, 23, 1008, 3250, 6339, 6340, 1129, 25, 2687], [95, 704, 136, 161, 151, 516, 89, 7, 179, 22, 4116, 380, 60, 682, 60, 49, 866, 3251, 642, 1509, 734, 616, 1130, 616, 1130, 471, 2024, 617, 683, 643, 642, 96, 174, 4117, 4118, 705, 666, 299, 245], [3252, 2303, 1510, 2, 22, 22, 40, 82, 6341, 2303, 6342, 6343, 6344, 3253, 6345, 178, 6346, 3252, 6347, 148, 6348, 79, 4119, 6349, 6350, 6351, 6352, 6353, 83, 6354, 1303, 572, 4120, 6355, 1131, 1009, 171, 6356, 2688, 180, 6357, 2689, 2688, 29, 442, 305, 666], [454, 1511, 1132, 181, 338, 454, 1511, 1132, 306, 251, 181, 338, 179, 30, 6358, 141, 40, 6359, 21, 84], [1304, 2025, 281, 828, 552, 181, 251, 262, 573, 226, 263, 314, 6360, 251, 226, 6361, 40, 56, 380, 60, 682, 60, 6362, 642, 734

In [64]:
# Load GloVe embeddings
embeddings_index = {}
with open('glove.6B.200d.txt', 'r', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs
 
# Create an embedding matrix
embedding_matrix = np.zeros((len(tokenizer.word_index) + 1, 200))
for word, i in tokenizer.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector
 
# Model building
model = Sequential()
model.add(Embedding(len(tokenizer.word_index) + 1, 200, weights=[embedding_matrix], input_length=250, trainable=False))
model.add(SpatialDropout1D(0.2))
model.add(Bidirectional(LSTM(200, dropout=0.1, recurrent_dropout=0.1)))
model.add(Dense(len(Y[0]), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
 
# Training the model
epochs = 6
batch_size =64
history = model.fit(X_train, Y_train, epochs=epochs, batch_size=batch_size,
                    validation_split=0.1, callbacks=[EarlyStopping(monitor='val_loss', patience=3, min_delta=0.0001)])

 
# Evaluate the model
test_loss, test_accuracy = model.evaluate(X_test, Y_test, verbose=1)
print(f'Test Accuracy: {test_accuracy:.3f}')

Epoch 1/7
17/17 [==============================] - 303s 17s/step - loss: 1.7796 - accuracy: 0.3972 - val_loss: 1.5941 - val_accuracy: 0.4661
Epoch 2/7
17/17 [==============================] - 311s 18s/step - loss: 1.5525 - accuracy: 0.4701 - val_loss: 1.3349 - val_accuracy: 0.5508
Epoch 3/7
17/17 [==============================] - 315s 19s/step - loss: 1.3289 - accuracy: 0.5630 - val_loss: 1.2674 - val_accuracy: 0.5678
Epoch 4/7
17/17 [==============================] - 309s 18s/step - loss: 1.2413 - accuracy: 0.5773 - val_loss: 1.2326 - val_accuracy: 0.5508
Epoch 5/7
17/17 [==============================] - 301s 18s/step - loss: 1.1644 - accuracy: 0.5991 - val_loss: 1.2705 - val_accuracy: 0.5339
Epoch 6/7
17/17 [==============================] - 311s 18s/step - loss: 1.1030 - accuracy: 0.6199 - val_loss: 1.2019 - val_accuracy: 0.5593
Epoch 7/7
16/16 [==============================] - 25s 2s/step - loss: 1.1908 - accuracy: 0.5952
Test Accuracy: 0.595


In [42]:
# Assuming X_test is already preprocessed and in the correct shape
predicted_probabilities = model.predict(X_test)

# Output the probabilities
print(predicted_probabilities[:10])


16/16 [==============================] - 31s 2s/step
[[0.00464838 0.00864111 0.48219764 0.32865733 0.0103936  0.13694334
  0.01812652 0.01039209]
 [0.00255645 0.00850599 0.63979876 0.22122972 0.00309161 0.09873965
  0.02088662 0.00519119]
 [0.00338048 0.00878909 0.79722    0.09432835 0.00771754 0.06786957
  0.0139389  0.00675601]
 [0.02813753 0.01154436 0.07134236 0.01501237 0.06705531 0.11031748
  0.6880391  0.0085514 ]
 [0.00095452 0.01960829 0.8988905  0.02676952 0.00690953 0.03652749
  0.00663387 0.00370631]
 [0.00166576 0.00774372 0.73713833 0.11932838 0.00349099 0.10859381
  0.01778415 0.00425474]
 [0.00400054 0.6294545  0.215172   0.01159432 0.04335735 0.03499484
  0.05117939 0.01024704]
 [0.00290792 0.44338736 0.26079246 0.01901368 0.04739543 0.10029608
  0.11909553 0.00711151]
 [0.00237431 0.02304557 0.5176224  0.25352824 0.01377222 0.15087934
  0.02900996 0.00976803]
 [0.03796301 0.01296554 0.02726889 0.01395181 0.16047542 0.14537486
  0.5759396  0.02606084]]


In [44]:
categories=data['Label'].unique()
print(categories)

['Health Equipments' 'Skin Care & Makeup' 'Other Health Care'
 'Vision Care' 'Fragrances' 'Hair Care' 'Medications & Supplements'
 'Other Beauty']


In [46]:
# Get predicted class indices
predicted_classes = predicted_probabilities.argmax(axis=-1)

# Map indices to names
predicted_category_names = [categories[i] for i in predicted_classes]

# Print the categories
print(predicted_category_names[:10])


['Other Health Care', 'Other Health Care', 'Other Health Care', 'Medications & Supplements', 'Other Health Care', 'Other Health Care', 'Skin Care & Makeup', 'Skin Care & Makeup', 'Other Health Care', 'Medications & Supplements']


In [50]:
i=1
# Print the test instance (depends on how X_test is structured)
print(f"Test Instance {i}: {X_test[i]}")
print(f"label:{i}:{Y_test[i]}")
    # Print the predicted probabilities
print(f"Predicted Probabilities: {predicted_probabilities[i]}")

    # Find the category with the highest probability
highest_prob_index = np.argmax(predicted_probabilities[i])
highest_prob_category = categories[highest_prob_index]

    # Print the category with the highest probability
print(f"Category with Highest Probability: {highest_prob_category}\n")

Test Instance 1: [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0

16/16 [==============================] - 29s 2s/step


TypeError: 'numpy.int32' object is not iterable

[nltk_data] Downloading package stopwords to C:\Users\Zeeshan
[nltk_data]     Gilani\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Epoch 1/5
17/17 [==============================] - 75s 4s/step - loss: 1.8282 - accuracy: 0.3829 - val_loss: 1.8102 - val_accuracy: 0.3814
Epoch 2/5
17/17 [==============================] - 72s 4s/step - loss: 1.6471 - accuracy: 0.4427 - val_loss: 1.7723 - val_accuracy: 0.3814
Epoch 3/5
17/17 [==============================] - 69s 4s/step - loss: 1.5491 - accuracy: 0.4616 - val_loss: 1.7347 - val_accuracy: 0.3983
Epoch 4/5
17/17 [==============================] - 69s 4s/step - loss: 1.4308 - accuracy: 0.4948 - val_loss: 1.6366 - val_accuracy: 0.3983
Epoch 5/5
16/16 [==============================] - 7s 422ms/step - loss: 1.5450 - accuracy: 0.4762
Test Accuracy: 0.476
